# RNN 기반 텍스트 생성 (언어모델)

In [1]:
corpus = ['경마장에 있는 말이 뛰고 있다',
        '그의 말이 법이다',
        '가는 말이 고와야 오는 말이 곱다']

## 1. 데이터 전처리

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer

# tokenizing해서 indexing -> 단어 수 확인
tokenizer = Tokenizer();
tokenizer.fit_on_texts(corpus)
vocab_size = len(tokenizer.word_index)
max_feature = vocab_size + 1
print(f'단어의 크기 : {vocab_size}')
print(f'max_feature : {max_feature}')

단어의 크기 : 11
max_feature : 12


In [13]:
# 언어 모델용 학습 데이터 만들기
sequences = list()

for sent in corpus:
    indexed_sent = tokenizer.texts_to_sequences([sent])[0] # integer encoding
    print(f'전체 문장 : {indexed_sent}')
    
    # 두번째 단어까지 시작해서 한 단어씩 추가, 학습 데이터 생성
    for i in range(1, len(indexed_sent)):
        sequence = indexed_sent[:i+1]
        print(sequence)
        sequences.append(sequence)
print(f'학습데이터 수 : {len(sequences)}')

전체 문장 : [2, 3, 1, 4, 5]
[2, 3]
[2, 3, 1]
[2, 3, 1, 4]
[2, 3, 1, 4, 5]
전체 문장 : [6, 1, 7]
[6, 1]
[6, 1, 7]
전체 문장 : [8, 1, 9, 10, 1, 11]
[8, 1]
[8, 1, 9]
[8, 1, 9, 10]
[8, 1, 9, 10, 1]
[8, 1, 9, 10, 1, 11]
학습데이터 수 : 11


In [15]:
# 모든 샘플에서 길이가 가장 긴 샘플의 길이 구하기
maxlen = 0
for sequence in sequences:
    if len(sequence) > maxlen:
        maxlen = len(sequence)

# maxlen = max([len(sequence) for sequence in sequences])

print(maxlen)

6


In [17]:
# 학습 데이터 padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_sequences = pad_sequences(sequences, maxlen=maxlen)
print(padded_sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [29]:
# 입력 데이터와 정답 데이터 분리
import numpy as np
np_sequences = np.array(padded_sequences)
X, y = np_sequences[:, :-1], np_sequences[:, -1]
print(X)
print(y)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]
[ 3  1  4  5  1  7  1  9 10  1 11]


In [31]:
# 정답 데이터 단어 11개를 카테고리로 one-hot-encoding
from tensorflow.keras.utils import to_categorical
y = to_categorical(y)
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 2. 모델 학습

In [39]:
# 모델 구축
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

input_units = max_feature
embedding_dim = 10
rnn_units = 32
output_units = max_feature

model = Sequential()
model.add(Embedding(input_units, embedding_dim)) # maxlen=max_len 생략
model.add(SimpleRNN(rnn_units))
model.add(Dense(output_units, activation='softmax'))
model.summary()

# rnn_lang_model = [
#     Embedding(input_units, embedding_dim),
#     SimpleRNN(rnn_units),
#     Dense(output_units, activation='softmax')
# ]

# model = Sequential(rnn_lang_model)
# model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 10)          120       
                                                                 
 simple_rnn (SimpleRNN)      (None, 32)                1376      
                                                                 
 dense (Dense)               (None, 12)                396       
                                                                 
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [41]:
# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
# 모델 학습
model.fit(X, y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 2.4967 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 28ms/step - loss: 2.4853 - accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 22ms/step - loss: 2.4740 - accuracy: 0.0909
Epoch 4/200
1/1 [==============================] - 0s 18ms/step - loss: 2.4627 - accuracy: 0.0909
Epoch 5/200
1/1 [==============================] - 0s 15ms/step - loss: 2.4514 - accuracy: 0.0909
Epoch 6/200
1/1 [==============================] - 0s 14ms/step - loss: 2.4401 - accuracy: 0.0909
Epoch 7/200
1/1 [==============================] - 0s 18ms/step - loss: 2.4286 - accuracy: 0.0909
Epoch 8/200
1/1 [==============================] - 0s 16ms/step - loss: 2.4170 - accuracy: 0.1818
Epoch 9/200
1/1 [==============================] - 0s 18ms/step - loss: 2.4051 - accuracy: 0.2727
Epoch 10/200
1/1 [==============================] - 0s 19ms/step - loss: 2.3929 - accuracy: 0.4545
Epoch 11/200


## 3. 첫 단어 입력 후 문장 생성

In [97]:
# 단어를 입력하고, 반복할 회수를 주면 문장 생성
# 모델, 토크나이저, 현재 단어, 반복할 횟수
def sentence_generation(model, tokenizer, init_word, n): 
    current_word = init_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        # Integer Encoding
        sequence = tokenizer.texts_to_sequences([current_word])
        
        # Padding
        padded_sequence = pad_sequences(sequence, maxlen=maxlen-1)

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(padded_sequence, verbose=0)
        result_index = np.argmax(result)
        print(result_index)
        
        # 예측한 인덱스의 단어 가져오기
        word = tokenizer.index_word[result_index]
        print(word)

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' ' +  word

    return current_word

In [99]:
sentence_generation(model, tokenizer, '경마장에', 4)

3
있는
1
말이
4
뛰고
5
있다


'경마장에 있는 말이 뛰고 있다'

In [101]:
sentence_generation(model, tokenizer, input('시작 단어 입력 : '), int(input('생성 단어 수 :')))

시작 단어 입력 :  가는
생성 단어 수 : 5


1
말이
9
고와야
10
오는
1
말이
11
곱다


'가는 말이 고와야 오는 말이 곱다'

In [55]:
# 단어를 입력하고, 반복할 회수를 주면 문장 생성
# 모델, 토크나이저, 현재 단어, 반복할 횟수
# def sentence_generation(model, tokenizer, init_word, n): 
#     current_word = init_word
#     sentence = ''

#     # n번 반복
#     for _ in range(n):
#         # Integer Encoding
#         encoded = tokenizer.texts_to_sequences([current_word])[0]
#         # Padding
#         padded = pad_sequences([encoded], maxlen=maxlen-1) #, padding='pre')

#         # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
#         result = model.predict(padded, verbose=0)
#         #print(result)
#         result_index = np.argmax(result, axis=1)

#         # 예측한 인덱스의 단어 가져오기
#         for word, index in tokenizer.word_index.items(): 
#             # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
#             if index == result_index:                break

#         # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
#         current_word = current_word + ' '  + word

#         # 예측 단어를 문장에 저장
#         sentence = sentence + ' ' + word

#     sentence = init_word + sentence
#     return sentence

# sentence_generation(model, tokenizer, '경마장에', 4)

'경마장에 있는 말이 뛰고 있다'